In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
import io
from PIL import Image
import PIL
import hashlib



In [2]:
import selenium
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = 'D:\Adelaide Study Details\Sem-1\Research\CLIP Algorithm research\chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_name = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
                    image_name.add(actual_image.get_attribute('alt'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls,image_name

In [4]:
def persist_image(folder_path:str,url:str,img_name:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        #file_path = os.path.join(folder_path,img_name + '.jpg')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10].replace(
            hashlib.sha1(image_content).hexdigest()[:10], img_name) + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
        


In [5]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res,name = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1)
       
    for elem,name1 in zip(res,name):
        persist_image(target_folder,elem,name1.replace(" ", "_"))
    return "Scraping completed"

In [ ]:
search_term_dog = ['dog','dog in house','small dog','big dog','furry dog','cute dog','happy dog','sitting dog','dancing dog',
                   'good dog']

for search_term in search_term_dog:

    search_and_download(search_term=search_term,driver_path=DRIVER_PATH,number_images = 1)



Found: 100 search results. Extracting links from 0:100
Found: 25 image links, done!
ERROR - Could not save https://akm-img-a-in.tosshub.com/indiatoday/images/story/202108/international_dog_day_2021_4_r_1200x768.jpeg?mhENil.rEsB2Wju30UDroUYKmJ4NfkX4&size=770:433 - [Errno 22] Invalid argument: './images\\dog\\Planning_on_bringing_home_a_dog?_Make_sure_you_pick_the_right_one_|__Lifestyle_News,The_Indian_Express.jpg'
SUCCESS - saved https://post.medicalnewstoday.com/wp-content/uploads/sites/3/2020/02/322868_1100-800x825.jpg - as ./images\dog\12_Common_Dog_Breeds_and_Their_Health_Issues.jpg
SUCCESS - saved https://www.cdc.gov/healthypets/images/pets/cute-dog-headshot.jpg?_=42445 - as ./images\dog\Dogs.jpg
SUCCESS - saved https://www.cdc.gov/healthypets/images/pets/angry-dog.jpg?_=03873 - as ./images\dog\How_canines_capture_your_heart:_scientists_explain_puppy_dog_eyes_|_Animal__behaviour_|_The_Guardian.jpg
SUCCESS - saved https://i.natgeofe.com/n/4f5aaece-3300-41a4-b2a8-ed2708a0a27c/domesti

ERROR - Could not save https://blissmaidservices.com/wp-content/uploads/How-To-Keep-Your-House-Clean-With-A-Dirty-Dog-01.jpg - [Errno 22] Invalid argument: './images\\dog_in_house\\10_Architects_Create_Stylish_Doghouses—for_a_Good_Cause_|_Architectural__Digest.jpg'
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWZjTRf7NSLrJtCqn7w0E8lOIgI0_hSRBQnw&usqp=CAU - as ./images\dog_in_house\Cartoon_Dog_House_Stock_Vector_Image_&amp;_Art_-_Alamy.jpg
ERROR - Could not save https://littledogtips.com/wp-content/uploads/2017/11/20171117_171019-e1511672548451.jpg - [Errno 22] Invalid argument: './images\\dog_in_house\\Dog_House_at_Rs_2500/piece_|_Dog_Shelter_House,_कुत्ते_का_घर_-_Paws_N__Claws,_Surat_|_ID:_16617490355.jpg'


In [ ]:
search_term_cat = ['cat','cat in house','small cat','big cat','furry cat','cute cat','happy cat','sitting cat','dancing cat',
                   'good cat']

for search_term in search_term_cat:
    search_and_download(search_term=search_term,driver_path=DRIVER_PATH,number_images = 2
    )

## Images data
* Dog Images - 3000
* Cat Images - 3000

### Source code reference :- https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

**Note:** Still working to correct the algorithm as it crashes after scraping around 50 images